In [43]:
from cirq import Circuit, H, CX, M, R, X, Y, Z, ResetChannel, CircuitOperation, LineQubit, depolarize
from cirq import Simulator, TOFFOLI, Condition, FrozenCircuit, flatten_to_ops, flatten_op_tree, ClassicallyControlledOperation
import abc, time
from qsimcirq import QSimCircuit, QSimSimulator
from cirq.value import MeasurementKey
from sympy import symbols, Eq

sim=QSimSimulator()
d=LineQubit.range(9)
a=LineQubit.range(9, 14)
encoder= Circuit(
    # Phase-flip code
    CX(d[0], d[3]),
    CX(d[3], d[6]),

    H.on_each(d[0:9:3]),

    # Bit-flip code
    CX(d[0], d[1]),
    CX(d[0], d[2]),

    CX(d[3], d[4]),
    CX(d[3], d[5]),

    CX(d[6], d[7]),
    CX(d[6], d[8]), )

error_channel=Circuit(
    depolarize(.01, 1).on_each(*d),
)

check_x_basis = Circuit(
    # Disentangle bit-flip code
    CX(d[0], d[2]),
    CX(d[0], d[1]),

    CX(d[3], d[5]),
    CX(d[3], d[4]),

    CX(d[6], d[8]),
    CX(d[6], d[7]),
    H.on_each(d[0:9:3]),

    # Parity on phase 
    CX(d[0], a[0]),
    CX(d[3], a[0]),
    CX(d[3], a[1]),
    CX(d[6], a[1]),

    M(a[:2], key='X'),
    ResetChannel().on_each(a[:2]),
    Z(d[0]).with_classical_controls(Eq(symbols('X'), 2)),
    Z(d[3]).with_classical_controls(Eq(symbols('X'), 3)),
    Z(d[6]).with_classical_controls(Eq(symbols('X'), 1)),
)

recover_x_basis = Circuit(
    # Reencode Bit-flip code
    H.on_each(d[0:9:3]),
    CX(d[0], d[1]),
    CX(d[0], d[2]),

    CX(d[3], d[4]),
    CX(d[3], d[5]),

    CX(d[6], d[7]),
    CX(d[6], d[8]), 
)


check_and_recover_z_basis = Circuit()
for j,k,i in zip(d[::3], a[::2], range(3)): 
    key=f'Z_{i}'
    check_and_recover_z_basis.append((
        CX(j,k),
        CX(j+1, k),
        CX(j+1, k+1),
        CX(j+2, k+1),
        M(*[k, k+1], key=key),
        ResetChannel().on_each(*[k, k+1]),

        X(j).with_classical_controls(Eq(symbols(key), 2)),
        X(j+1).with_classical_controls(Eq(symbols(key), 3)),
        X(j+2).with_classical_controls(Eq(symbols(key), 1)),
    ))


decoder = Circuit(
    CX(d[0], d[1]),
    CX(d[0], d[2]),

    CX(d[3], d[4]),
    CX(d[3], d[5]),

    CX(d[6], d[7]),
    CX(d[6], d[8]),
    TOFFOLI(d[2], d[1], d[0]),
    TOFFOLI(d[5], d[4], d[3]),
    TOFFOLI(d[8], d[7], d[6]),
    H.on_each(d[0:9:3]),

    CX(d[0], d[3]),
    CX(d[3], d[6]),

    TOFFOLI(d[6], d[3], d[0]),

    M(d[0], key='Psi')
)
Psi=Circuit()
total_circ=Psi+encoder+error_channel+check_x_basis+recover_x_basis+check_and_recover_z_basis+decoder
total_circ_compiled = QSimCircuit(total_circ)
sim.sample(total_circ_compiled, repetitions=1).sum()

ValueError: Encountered unparseable op: Z(q(0)).with_classical_controls(Eq(X, 2))

In [48]:
sim=QSimSimulator(circuit_memoization_size=2)
QC=Circuit(
    H(d[0]),
    CX(d[0], d[1]),
    M(d[1], key='m'),
    X(d[0]).with_classical_controls('m')
)
    

sim.run(QSimCircuit(QC))


ValueError: Encountered unparseable op: X(q(0)).with_classical_controls(m)